# ph recognition based on the image color
 ***
 
 ### I downloaded ph-scale images from [link](https://www.shutterstock.com/search/ph+scale). Then I split every image to 15 small ones and I labeled it based on the ph

In [26]:
import cv2 # for images
import matplotlib.pyplot as plt
import pickle
import numpy as np
import pandas as pd
from sklearn.utils import shuffle
from sklearn import svm
from sklearn.externals import joblib

pd.set_option('display.max_columns', None)

data_dir = 'C:\\Users\\asus\\Desktop\\data_science_python\\ph_project\\ph_datasets\\'

In [27]:
PH_COUNT = 15
images_list = []
PH_DATASETS = 7
# add image to list in format [blue, green, red, label=ph]
for j in range(PH_DATASETS):
    for i in range(PH_COUNT):
        image = cv2.imread(data_dir + 'ph{}\\{}.jpg'.format(j+1, i))
        images_list.append([image[3][3][0],image[3][3][1],image[3][3][2],i])

# transform data to pandas DataFrame
df = pd.DataFrame(np.asarray(images_list),columns=['blue','green','red','label'])
df = shuffle(df)
df.head()

,blue,green,red,label
68,84,166,0,8
32,17,198,248,2
67,0,153,0,7
61,27,83,255,1
55,200,144,73,10


In [13]:
y = np.array(df['label']) 
X = np.array(df.drop(['label'], axis=1))
# print(X)

# test and train data 

In [14]:
SIZE = X.shape[0]
TRAIN_SIZE = int(SIZE*0.8)
train_x = X[:TRAIN_SIZE]
train_y = y[:TRAIN_SIZE]
test_x = X[TRAIN_SIZE:]
test_y = y[TRAIN_SIZE:]

# Testing machine learning algorithms

***

 svm -> very bad results

decision tree -> accuracy 0.57%, add 1 point mistake -> accuracy 0,95% 

gredient boosting classifier -> accuracy 0.47%

In [53]:
from sklearn import tree
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor()
clf.fit(train_x, train_y)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=3, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=2, min_weight_fraction_leaf=0.0,
             n_estimators=100, presort='auto', random_state=None,
             subsample=1.0, verbose=0, warm_start=False)

In [54]:
# testing 
accuracy = 0
for i in range(len(test_x)):
    y_predict = clf.predict([test_x[i]])
    print('Predict {}, labels {}'.format(y_predict[0],test_y[i]))
    # with 1 margin point
    if(abs(y_predict[0] - test_y[i]) <= 0.5):
        accuracy += 1
accuracy = float(accuracy/len(test_x))
print(accuracy)
# return the mean accuracy on the given test data and labels
print(clf.score(test_x, test_y))

Predict 6.022316133377997, labels 6
Predict 6.179363820970485, labels 6
Predict 12.397021877053094, labels 12
Predict 0.23262833771536454, labels 0
Predict 13.223374768560294, labels 13
Predict 8.995948477022658, labels 9
Predict 6.022316133377997, labels 6
Predict 8.995948477022658, labels 8
Predict 11.658411342660076, labels 12
Predict 0.006063371049064589, labels 0
Predict 0.7769754093019112, labels 1
Predict 10.976236003373401, labels 11
Predict 0.9374502131359876, labels 1
Predict 10.002864379205464, labels 9
Predict 3.720628855574921, labels 3
Predict 6.992665174748736, labels 7
Predict 2.0120518107750787, labels 2
Predict 11.664674588625694, labels 12
Predict 0.9374502131359876, labels 1
Predict 2.3696431544259844, labels 2
Predict 6.172626284334054, labels 6
0.8571428571428571
0.9916893085533858


In [35]:
# save tree model 
joblib.dump(clf,'gradient_regression_85.pkl')

['gradient_regression_85.pkl']